<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Backpropagation-Practice" data-toc-modified-id="Backpropagation-Practice-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Backpropagation Practice</a></span><ul class="toc-item"><li><span><a href="#Data-Science-Unit-4-Sprint-2-Assignment-2" data-toc-modified-id="Data-Science-Unit-4-Sprint-2-Assignment-2-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span><em>Data Science Unit 4 Sprint 2 Assignment 2</em></a></span></li><li><span><a href="#Try-building/training-a-more-complex-MLP-on-a-bigger-dataset." data-toc-modified-id="Try-building/training-a-more-complex-MLP-on-a-bigger-dataset.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Try building/training a more complex MLP on a bigger dataset.</a></span></li><li><span><a href="#Stretch-Goals:" data-toc-modified-id="Stretch-Goals:-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Stretch Goals:</a></span></li></ul></li></ul></div>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
##### Your Code Here #####
import numpy as np

def create_test_data(num_of_samples=10000):
    """
    Function to create X and y test samples of user defined length.
    """
    
    # Create a vector of length 3 with each value being 1 or 0.
    X = np.array([np.random.randint(0, 2, 3) for _ in range(num_of_samples)])
    
    # Create vector of length 1 with value of 1 if sum of first two values from X vector sum to 1.
    # Otherwise value 0.
    # First two values in X vector match input to XOR Logic Gate and its output. 
    y = np.array([1 if data[0] + data[1] == 1 else 0 for data in X])
    y = y.reshape(len(y), 1)
    
    return X, y

In [2]:
# Create test data.

X, y = create_test_data()

In [3]:
# Use the base NeuralNetwork Class from the lecture, with 

class NeuralNetwork: 
    
    def __init__(self):
        # Setup Arch
        self.inputs = 3
        self.hiddenNodes = 4 
        self.outputNodes = 1
        
        # Initialize Weights
        # 3x4
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
        
        # 4x1
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        sx = self.sigmoid(s)
        return sx * (1 - sx)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        
        # Weight Sum
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activation
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weighted Sum 2
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final Output
        self.activated_output = self.sigmoid(self.output_sum)
        # Reformatting the Final output to round the guess to 0 and 1.
        output_formatted = np.array([0 if result <= 0.50 else 1 for result in self.activated_output])
        output_formatted = output_formatted.reshape(len(self.activated_output), 1)
        
        return output_formatted
        
    def backward(self, X, y, o):
        """
        Back prop through the network
        """
        
        self.o_error = y - o
        
        # Apply derivative of sigmoid to error
        self.o_delta = self.o_error * self.sigmoidPrime(self.output_sum)
        
        # z2 error: how much were our output layer weights off
        self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # z2 delta: how much were the weights off?
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.hidden_sum)

        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [ ]:
# Train my 'net
nn = NeuralNetwork()

# Number of Epochs / Iterations
for i in range(10000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 1000 == 0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---' * 3 + '+')
        print('Input: \n', X[:5])
        print('Actual Output: \n', y[:5])
        print('Predicted Output: \n', str(nn.feed_forward(X)[:5]))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X, y)

+---------EPOCH 1---------+
Input: 
 [[1 0 1]
 [0 1 1]
 [1 0 0]
 [0 0 0]
 [0 1 0]]
Actual Output: 
 [[1]
 [1]
 [1]
 [0]
 [1]]
Predicted Output: 
 [[0]
 [0]
 [0]
 [0]
 [0]]
Loss: 
 0.5013
+---------EPOCH 2---------+
Input: 
 [[1 0 1]
 [0 1 1]
 [1 0 0]
 [0 0 0]
 [0 1 0]]
Actual Output: 
 [[1]
 [1]
 [1]
 [0]
 [1]]
Predicted Output: 
 [[1]
 [1]
 [1]
 [1]
 [1]]
Loss: 
 0.4987
+---------EPOCH 3---------+
Input: 
 [[1 0 1]
 [0 1 1]
 [1 0 0]
 [0 0 0]
 [0 1 0]]
Actual Output: 
 [[1]
 [1]
 [1]
 [0]
 [1]]
Predicted Output: 
 [[1]
 [1]
 [1]
 [1]
 [1]]
Loss: 
 0.4987
+---------EPOCH 4---------+
Input: 
 [[1 0 1]
 [0 1 1]
 [1 0 0]
 [0 0 0]
 [0 1 0]]
Actual Output: 
 [[1]
 [1]
 [1]
 [0]
 [1]]
Predicted Output: 
 [[1]
 [1]
 [1]
 [1]
 [1]]
Loss: 
 0.4987
+---------EPOCH 5---------+
Input: 
 [[1 0 1]
 [0 1 1]
 [1 0 0]
 [0 0 0]
 [0 1 0]]
Actual Output: 
 [[1]
 [1]
 [1]
 [0]
 [1]]
Predicted Output: 
 [[1]
 [1]
 [1]
 [1]
 [1]]
Loss: 
 0.4987


## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [ ]:
##### Your Code Here #####

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?